We are going to use imdb dataset to create content based recommended system.

Datasets = https://drive.google.com/file/d/1bPm6emwAUgCwGs_jX6FGAUTPUCDcvApR/view?usp=sharing,%20https://drive.google.com/file/d/1urB8DUCOEMAbMyo1sMBTkcKm7JquZ2dE/view?usp=sharing



In [ ]:
import pandas as pd
import numpy as np 
credits = pd.read_csv("/content/tmdb_5000_credits.csv")
movies = pd.read_csv("/content/tmdb_5000_movies.csv")
(movies.info())
(movies.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [ ]:
credits.info()
credits.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
print('credits :',credits.shape)
print('movies :',movies.shape)

credits : (4803, 4)
movies : (4803, 20)


Now we will merge both the datasets on the basis of movie id



In [ ]:
credits_renamed = credits.rename(index=str,columns={"movie_id":"id"})
credits_renamed = credits.drop(columns=['title'])
movies_credits_merge= movies.merge(credits_rename, on='id')
(movies_credits_merge.head())

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
movies_cleaned =movies_credits_merge.drop(columns=['homepage','status','title_x','title_y','production_countries'])
movies_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   id                    4803 non-null   int64  
 3   keywords              4803 non-null   object 
 4   original_language     4803 non-null   object 
 5   original_title        4803 non-null   object 
 6   overview              4800 non-null   object 
 7   popularity            4803 non-null   float64
 8   production_companies  4803 non-null   object 
 9   release_date          4802 non-null   object 
 10  revenue               4803 non-null   int64  
 11  runtime               4801 non-null   float64
 12  spoken_languages      4803 non-null   object 
 13  tagline               3959 non-null   object 
 14  vote_average          4803 non-null   float64
 15  vote_count           

Now we are going to use movie summaries to create a vector for each movie in which we will use the words in our summary. Then we will compare vectors of all movies and recommend those which will have similar values.

We will use tfidf from sklearn to implement this.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df = 3,
                      max_features=None,
                      strip_accents='unicode',
                      analyzer='word',
                      token_pattern= 'r\w{1,}', # \w{1,} = alphanum with minimum 1 letter
                      ngram_range=(1,3), # will create combination  sets of 1 to 3 words
                      stop_words='english')


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=3, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents='unicode',
                sublinear_tf=False, token_pattern='r\\w{1,}', tokenizer=None,
                use_idf=True, vocabulary=None)


In [ ]:
# we fill fit the tfidf on summary text
print(movies_cleaned['overview'])
tfv_matrix = tfv.fit_transform(movies_cleaned['overview'].values.astype('U')) #astype converts it to unicode
print(tfv_matrix, ' \n', tfv_matrix.shape)



0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object
  (0, 2597)	0.5703087350597803
  (0, 566)	0.4577234082379352
  (0, 2107)	0.28163582705549167
  (0, 0)	0.3591850534007457
  (0, 1631)	0.4710961561327209
  (0, 3455)	0.18701069026879158
  (1, 2152)	0.8264344445810327
  (1, 3174)	0.5630329553498978
  (2, 2921)	0.3636372449253235
  (2, 3421)	0.3636372449253235
  (2, 1189

In [58]:
# so now we have vectors 

from sklearn.metrics.pairwise import sigmoid_kernel

'''sigmoid kernel basically calculates hyperbolic tangent 
betw two vectors,i.e,distance between 2 vectors
This will show us how  similar two movie summaries are based 
on the distances betw vector values'''

sigma = sigmoid_kernel(tfv_matrix,tfv_matrix)
print(sig[0])

[0.76170995 0.76159416 0.76159416 ... 0.76159416 0.76159906 0.76159577]


In [57]:
# Now we will map these values with their titles 

# we have created a Series which has title:index which is hashed
indices = pd.Series(movies_cleaned.index, index = movies_cleaned['original_title']).drop_duplicates()

print(indices)
print(indices['Avatar'])
print(sig[4802])
print(list(enumerate(sigma[indices['Avatar']]))) # this shows us vector of avatar compared with all the movies
# now we have sorted the list based on vector values closest to our given movie
print(sorted(list(enumerate(sigma[indices['Avatar']])),key=lambda x: x[1], reverse=True))


original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64
0
[0.76159577 0.76159416 0.76159592 ... 0.76159569 0.76159489 0.76170995]
[(0, 0.7617099546704214), (1, 0.7615941559557649), (2, 0.7615941559557649), (3, 0.7615941559557649), (4, 0.7615977758616139), (5, 0.7615941559557649), (6, 0.7615941559557649), (7, 0.7615963409078871), (8, 0.7615992303956007), (9, 0.76159901150737), (10, 0.7615941559557649), (11, 0.7615941559557649), (12, 0.7615941559557649), (13, 0.76159415595

In [62]:
# now we will give create a function to give 10 recommendations based on movie title

def give_recommendations(title):
    #get index
    idx= indices[title]


    #Get similarity score
    sigma_scores = list(enumerate(sigma[idx]))

    #sort the movies according to scores
    sigma_scores = sorted(sigma_scores, key= lambda x: x[1], reverse=True)

    #get 10 most similar movies
    sigma_scores =sigma_scores[1:11]
   # print(sigma_scores)

    #get Movie indices
    movie_indices = [ i[0] for i in sigma_scores]
    #print(movie_indices)

    #print 10 similar movies
    print('Movies similar to ',title," are : \n")
    return movies_cleaned['original_title'].iloc[movie_indices]


print(give_recommendations('Spectre'),'\n')
print(give_recommendations("The Dark Knight Rises"))

# this was a very basic attempt at doing this , I will be fine tuning this...

Movies similar to  Spectre  are : 

3162                    Thunderball
1281                        Hackers
3351    The Man with the Golden Gun
1967                The Fisher King
367                 The Interpreter
4402            In Her Line of Fire
3285                       Restless
753                    The Sentinel
1200           The Living Daylights
2933                       F.I.S.T.
Name: original_title, dtype: object 

Movies similar to  The Dark Knight Rises  are : 

299                            Batman Forever
2507                                Slow Burn
65                            The Dark Knight
1181                                      JFK
879                       Law Abiding Citizen
1202                             Legal Eagles
3595    Halloween: The Curse of Michael Myers
307                         The Expendables 3
2193                     Secret in Their Eyes
42                                Toy Story 3
Name: original_title, dtype: object
